<a href="https://colab.research.google.com/github/visiont3lab/deep-learning-course/blob/main/colab/Regression_Classification_Examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pytorch Pipeline for Regression/Classification Problem


In questo notebook ci focalizzero sul definere una pipeline di training e testing usando la libreria di deep learning [Pytorch](https://pytorch.org/). 
L'obbittivo di questo notebook è di definire tutti gli step e funzioni necessarie alla realizzazzione sia di un modello di regressione si di classificazione.


## Concetti da ricordare

## Importa le librerie

In [123]:
from torch import nn
from torch.utils.data import DataLoader
from torch import optim
import torch
from torch import nn
from torchsummary import summary
#!pip install torchsummary
import torch.nn.functional as F
from torch.utils.data import TensorDataset,Dataset
from torchvision import datasets
from torchvision import transforms
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import numpy as np
import plotly.graph_objects as go
# Loss function pytorch: https://neptune.ai/blog/pytorch-loss-functions

import pandas as pd
from datetime import datetime

## Activation Function Numpy vs Pytorch

* **Softmax** $$S_{i,j}(x) = \frac{e^x_{i,j}}{\sum_{l=1}^{L}{e^{x_{i,l}}}}$$ . Essa definisce un valore di probabilità per ogni input (x). La somma di tali valori è uguale ad 1

In [2]:
# Softmax Function
def softmax(x):
  values = np.exp(x)
  probs = values / np.sum(values,axis=-1,keepdims=True)
  return probs

# Pytorch Softmax
x = torch.tensor([[3.4,5,6,0.5],[5.6,7,4,3]],dtype=torch.float32)
print( f"Pytorch Tensor Shape: {x.shape}")
print( f"Pytorch Softmax: {torch.log_softmax(x, dim=-1).tolist()}" )

# Numpy Softmax
x = x.numpy()
print( f"Numpy   Softmax: {np.log(softmax(x)).tolist()}" )

Pytorch Tensor Shape: torch.Size([2, 4])
Pytorch Softmax: [[-2.9689667224884033, -1.368966817855835, -0.36896687746047974, -5.868967056274414], [-1.6736083030700684, -0.2736082375049591, -3.2736082077026367, -4.273608207702637]]
Numpy   Softmax: [[-2.9689669609069824, -1.3689669370651245, -0.3689669966697693, -5.868967056274414], [-1.673608422279358, -0.2736082375049591, -3.2736082077026367, -4.273608207702637]]


## Neural Network Model definition (Regression / Classification)

* **RegressionNet** è una rete neurale costituida da 2 hidden layer rispettivamente di 6 e 4 neuroni e un output layer avente un neurone. La funzione di attivazione utilizzata negli hidden layer è una tangente iperbolica (è possibile utilizzare anche relu o sigmoid)
* **ClassificationNet** è una rete neurale costituida da 2 hidden layer rispettivamente di 6 e 4 neuroni e un output layer avente un neurone. E' importante sottolineare che un activation function di nome softmax è presente sull'output layer. La funzione di attivazione utilizzata negli hidden layer è la Rectified Linear Unit (Relu).
* **nn.Linear(1,6)** definisce un fully connected layer dove il numero di input è 1 e il numero di neuroni è 6. Di conseguenza da tale layer usciranno 6 input per il layer sucessivo.


In [16]:
# Definiamo la Rete per la regressione
class RegressionNet(nn.Module):
    def __init__(self,num_inputs):
        super(RegressionNet,self).__init__()
        self.fc1 = nn.Linear(num_inputs,50)
        self.fc2 = nn.Linear(50,10)
        self.fc3 = nn.Linear(10,1)
    def forward(self,x):
        # torch.sigmoid, torch.tanh, torch.relu
        x = torch.tanh(self.fc1(x)) 
        x = torch.tanh(self.fc2(x))
        x = self.fc3(x)
        return x

#x = torch.tensor([0.9],dtype=torch.float32)
#x = x.reshape(1,-1) 
#x = x.unsqueeze(0)

RN = RegressionNet(num_inputs=1)
x = torch.tensor([[0.9]],dtype=torch.float32)
y = RN.forward(x)
#y.detach().numpy()
#y.tolist()
print(y)

tensor([[-0.4436]], grad_fn=<AddmmBackward>)


In [48]:
# Definiamo la rete per la classificazione
class ClassificationNet(nn.Module):
    def __init__(self,num_inputs, num_classes):
        super(ClassificationNet,self).__init__()
        self.num_classes = num_classes
        self.fc1 = nn.Linear(num_inputs,6)
        self.fc2 = nn.Linear(6,4)
        self.fc3 = nn.Linear(4,self.num_classes)
    def forward(self,x):
        # torch.sigmoid, torch.tanh, torch.relu
        x = torch.relu(self.fc1(x)) 
        x = torch.relu(self.fc2(x))
        x = torch.log_softmax(self.fc3(x),dim=-1) # sarebbe dim=1  print(self.fc3(x)) print(self.fc3(x).sum(dim=-1))
        return x

CN = ClassificationNet(num_inputs=1, num_classes=4)
x = torch.tensor([[0.9]],dtype=torch.float32)
y = CN.forward(x)
print(f"Sum {y.sum(dim=-1).item()} -- Numpy array: {np.round(y.detach().numpy(),2)}")


Sum -5.6050262451171875 -- Numpy array: [[-1.34 -1.16 -1.64 -1.46]]


## Datasets Data Loader Creation (Regression / Classification)

* Regressione: Creare un Modello capaci di approssimare qualsiasi funzione (lineare o non lineare). Per esempio sin e coseno
* Classificazione: Creare un modello capace di predirre la zona di rischio (bianca, gialla,arancione, rossa) italiana. [Dataset](https://github.com/visiont3lab/project-work-ifoa/blob/main/data/dpc-covid19-ita-regioni-zone.csv)


### Creazione dataset Regressione

In [99]:
# Dati Numpy
R_X = np.linspace(-4,3,2000)
R_Y = np.exp(0.2*R_X)*np.sin(3*R_X) - np.cos(R_X)

# Dati Pytorch Tensor
R_Xt = torch.from_numpy(R_X).type(torch.float32).unsqueeze(1)
R_Yt = torch.from_numpy(R_Y).type(torch.float32).unsqueeze(1)
print(f"X Tensor data shape: ", R_Xt.shape)
print(f"Y Tensor data shape: ", R_Yt.shape)

# Training and Test Set
R_X_train, R_X_test, R_Y_train, R_Y_test = train_test_split(R_X,R_Y,test_size=0.3,shuffle=True,random_state=4)
print(f"X Train shape: {R_X_train.shape} , X Test shape: {R_X_test.shape}")

# Normalization
R_mean = np.mean(R_X)
R_std = np.std(R_X)

# Tensor Dataset Che converte i dati da numpy a Pytorch
class CustomTensorDataset(Dataset):
    def __init__(self, x,y,mean,std):
        x = (x - mean)/std
        self.x = torch.from_numpy(x).type(torch.float32).unsqueeze(1)
        self.y = torch.from_numpy(y).type(torch.float32).unsqueeze(1)
    def __getitem__(self, index):
        x = self.x[index]
        y = self.y[index]
        return x, y
    def __len__(self):
        return self.x.shape[0]

# Dataset generator creation
R_train_ds = CustomTensorDataset(R_X_train,R_Y_train,R_mean,R_std)
R_test_ds = CustomTensorDataset(R_X_test,R_Y_test,R_mean,R_std)
R_train_dl = DataLoader(R_train_ds,batch_size=10,shuffle=True)
R_test_dl = DataLoader(R_test_ds,batch_size=5,shuffle=True)

for x,y in R_train_dl:
  print(x,y)
  break

X Tensor data shape:  torch.Size([2000, 1])
Y Tensor data shape:  torch.Size([2000, 1])
X Train shape: (1400,) , X Test shape: (600,)
tensor([[-0.0736],
        [ 1.1752],
        [-1.5389],
        [ 1.5736],
        [-1.2895],
        [ 0.4945],
        [ 1.4731],
        [-1.3484],
        [ 0.8756],
        [-0.4079]]) tensor([[-1.6138],
        [-0.5865],
        [ 1.3710],
        [ 2.5748],
        [ 0.9438],
        [ 0.2246],
        [ 2.2873],
        [ 1.1280],
        [-1.0955],
        [ 0.3238]])


In [100]:
# Visualization
fig = go.Figure()
fig.add_traces( go.Scatter(x=R_X_train, y=R_Y_train,hovertemplate='x: %{x} <br>y: %{y}',mode="markers", name="Train data") )
fig.add_traces( go.Scatter(x=R_X_test, y=R_Y_test,hovertemplate='x: %{x} <br>y: %{y}',mode="markers", name="Test data") )

fig.update_layout(title="Funzione di stimare")
fig.show()


### Creazione dataset Classificazione

In [131]:
import warnings
warnings.filterwarnings('ignore')

# Classificazione
df = pd.read_csv("https://raw.githubusercontent.com/visiont3lab/project-work-ifoa/main/data/dpc-covid19-ita-regioni-zone.csv")
df["data"] = [ datetime.strptime(d, "%Y-%m-%d %H:%M:%S").date() for d in  df["data"]]
#df = df[df["denominazione_regione"]=="Emilia-Romagna"]
df = df[df["zona"]!="unknown"].copy()
# Semplicazione e scelta degli input
names = ["data","denominazione_regione","zona","ricoverati_con_sintomi","terapia_intensiva",
        "totale_ospedalizzati","totale_positivi","isolamento_domiciliare",
        "deceduti","dimessi_guariti","nuovi_positivi","totale_casi","tamponi"]
df = df[names]

df_X = df[["ricoverati_con_sintomi","terapia_intensiva","totale_ospedalizzati","totale_positivi","isolamento_domiciliare","deceduti","dimessi_guariti","nuovi_positivi","totale_casi","tamponi"]]
df_Y = df["zona"]


oneHot = pd.get_dummies(df["denominazione_regione"], prefix='R')
for k in oneHot.keys():
    df_X.loc[:,k] = oneHot[k]

display(df_X.head())

,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,totale_positivi,isolamento_domiciliare,deceduti,dimessi_guariti,nuovi_positivi,totale_casi,tamponi,R_Abruzzo,R_Basilicata,R_Calabria,R_Campania,R_Emilia-Romagna,R_Friuli Venezia Giulia,R_Lazio,R_Liguria,R_Lombardia,R_Marche,R_Molise,R_P.A. Bolzano,R_P.A. Trento,R_Piemonte,R_Puglia,R_Sardegna,R_Sicilia,R_Toscana,R_Umbria,R_Valle d'Aosta,R_Veneto
5376,468,42,510,8581,8071,584,4340,395,13505,308505,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5377,100,16,116,2335,2219,59,778,249,3172,112980,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5378,212,15,227,4481,4254,132,2101,264,6714,292222,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5379,1677,180,1857,62196,60339,796,15017,4508,78009,1075201,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5380,1673,177,1850,33730,31880,4752,28559,1953,67041,1695309,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [83]:
# Dati Numpy
C_X = df_X.values
dict_names = {"bianca":0,"gialla": 1, "arancione": 2, "rossa": 3}
C_Y = np.array([dict_names[d] for d in df_Y],dtype=np.float) #.reshape(-1,1)
print(f"X shape: {C_X.shape} , Y shape: {C_Y.shape}")

# Training and Test Set
C_X_train, C_X_test, C_Y_train, C_Y_test = train_test_split(C_X,C_Y,test_size=0.3,shuffle=True,random_state=2)
print(f"X Train shape: {C_X_train.shape} , X Test shape: {C_X_test.shape}")

# Normalization
C_mean = np.mean(C_X)
C_std = np.std(C_X)
C_min = np.min(C_X)
C_max = np.max(C_X)

# Tensor Dataset Che converte i dati da numpy a Pytorch
class CustomTensorDataset(Dataset):
    def __init__(self, x,y,mean,std):
        x = (x - mean)/std           # Standard Scaler 
        #x = (x - min) / (max - min) # Min Max Scaler
        self.x = torch.from_numpy(x).type(torch.float32)
        self.y = torch.from_numpy(y).type(torch.LongTensor)
    def __getitem__(self, index):
        x = self.x[index]
        y = self.y[index]
        return x, y
    def __len__(self):
        return self.x.shape[0]

# Dataset generator creation
C_train_ds = CustomTensorDataset(C_X_train,C_Y_train,C_mean,C_std)
C_test_ds = CustomTensorDataset(C_X_test,C_Y_test,C_mean,C_std)

# Data Loader Creation
C_train_dl = DataLoader(C_train_ds,batch_size=64,shuffle=True)
C_test_dl = DataLoader(C_test_ds,batch_size=64,shuffle=True)

X shape: (2688, 31) , Y shape: (2688,)
X Train shape: (1881, 31) , X Test shape: (807, 31)


## Training 

In [129]:
# Validation: Metric Regression
def metrics_func_regression(target, output):
  # Comptue mean squaer error (Migliora quanto piu' ci avviciniamo a zero)
  mse = torch.sum((output - target) ** 2)
  return mse

# Validation: Metric cassification
def metrics_func_classification(target, output):
  # Compute number of correct prediction
  pred = output.argmax(dim=-1,keepdim=True)
  corrects =pred.eq(target.reshape(pred.shape)).sum().item()
  return -corrects # minus for coeherence with best result is the most negative one

# Training: Loss calculation and backward step
def loss_batch(loss_func,metric_func, xb,yb,yb_h, opt=None):
  # obtain loss
  loss = loss_func(yb_h, yb)
  # obtain permormance metric 
  metric_b = metric_func(yb,yb_h)
  if opt is not None:
    loss.backward()
    opt.step()
    opt.zero_grad()
  return loss.item(), metric_b

# Trainig: Function 1 epoch
def loss_epoch(model, loss_func,metric_func, dataset_dl, opt, device):
  loss = 0.0
  metric = 0.0
  len_data = len(dataset_dl.dataset)
  # Get batch data
  for xb,yb in dataset_dl:    
    # Send to cuda the data (batch size)
    xb = xb.to(device)
    yb = yb.to(device)
    # obtain model output 
    yb_h = model.forward(xb)
    # Loss and Metric Calculation
    loss_b, metric_b = loss_batch(loss_func,metric_func, xb,yb,yb_h,opt)
    loss += loss_b
    if metric_b is not None:
      metric+=metric_b 
  loss /=len_data
  metric /=len_data
  return loss, metric

# Training: Iterate on epochs
def train_val(epochs, model, loss_func, metric_func, opt, train_dl,test_dl,device, path2weigths="./weights.pt"):
  lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(opt, gamma=0.999) #  lr = lr * gamma ** last_epoch
  best_val_metric = 1000000
  for epoch in range(epochs):
    model.train()
    train_loss,train_metric = loss_epoch(model, loss_func, metric_func,train_dl, opt,device)
    lr_scheduler.step()
    model.eval()
    with torch.no_grad():
      val_loss, val_metric = loss_epoch(model, loss_func, metric_func, test_dl,opt=None,device=device)
      print("epoch: %d, train_loss: %.6f, val loss: %.6f,  train_metric: %.3f test_metric: %.3f lr: %.5f)" % (epoch,train_loss, val_loss,train_metric,val_metric,opt.param_groups[0]['lr']))
      if (val_metric <= best_val_metric):        
        # Save Models (It save last weights)
        torch.save(model.state_dict(),path2weigths)
        best_val_metric = val_metric


In [ ]:
class ClassificationNet(nn.Module):
    def __init__(self,num_inputs, num_classes):
        super(ClassificationNet,self).__init__()
        self.num_classes = num_classes
        self.fc1 = nn.Linear(num_inputs,200)
        self.fc2 = nn.Linear(200,100)
        self.fc3 = nn.Linear(100,50)
        self.fc4 = nn.Linear(50,self.num_classes)
    def forward(self,x):
        # torch.sigmoid, torch.tanh, torch.relu
        x = torch.tanh(self.fc1(x)) 
        x = torch.tanh(self.fc2(x))
        x = torch.tanh(self.fc3(x))
        x = torch.log_softmax(self.fc4(x),dim=-1) # sarebbe dim=1  print(self.fc3(x)) print(self.fc3(x).sum(dim=-1))
        return x

class RegressionNet(nn.Module):
    def __init__(self,num_inputs):
        super(RegressionNet,self).__init__()
        self.fc1 = nn.Linear(num_inputs,20)
        self.fc2 = nn.Linear(20,10)
        self.fc3 = nn.Linear(10,1)
    def forward(self,x):
        # torch.sigmoid, torch.tanh, torch.relu
        x = torch.tanh(self.fc1(x)) 
        x = torch.tanh(self.fc2(x))
        x = self.fc3(x)
        return x

# Setup GPU Device
device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda:0")

# Regression
R_model = RegressionNet(num_inputs=1).to(device)
R_loss_func = nn.MSELoss(reduction="sum") 
R_opt = optim.Adam(R_model.parameters(),lr=0.001)
R_train_dl = DataLoader(R_train_ds,batch_size=10,shuffle=True)
R_test_dl = DataLoader(R_test_ds,batch_size=5,shuffle=True)

# Classification
C_model= ClassificationNet(num_inputs=31,num_classes=4).to(device) # 10 31
C_loss_func = nn.NLLLoss(reduction="sum")
C_opt = optim.Adam(C_model.parameters(),lr=0.001)
#C_opt = optim.SGD(C_model.parameters(), lr=0.001, momentum=0.9)
C_train_dl = DataLoader(C_train_ds,batch_size=124,shuffle=True)
C_test_dl = DataLoader(C_test_ds,batch_size=124,shuffle=True)
# Regression
#train_val(200,R_model,R_loss_func,metrics_func_regression,R_opt, R_train_dl,R_test_dl,device,path2weigths="./weights_regression.pt")

# Classification
train_val(3000, C_model, C_loss_func, metrics_func_classification,C_opt, C_train_dl, C_test_dl, device, path2weigths="./weights_classication.pt")

## Test Models

In [134]:
# Run on cpu
device = torch.device("cpu")

# Load Regression
R_model = RegressionNet(num_inputs=1).to(device)
R_weights = torch.load("weights_regression.pt")
R_model.load_state_dict(R_weights)
R_model = R_model.to(device)

# Predict Regression
R_Xt = torch.from_numpy(R_X).type(torch.float32).unsqueeze(1)
R_Xt = (R_Xt - R_mean) / R_std
R_Y_hat = R_model.forward(R_Xt).detach().numpy().reshape(-1)

# Visualize Regression
fig = go.Figure()
fig.add_traces( go.Scatter(x=R_X, y=R_Y, name="Real",hovertemplate='x: %{x} <br>y: %{y}') )
fig.add_traces( go.Scatter(x=R_X, y=R_Y_hat, name="Predicted",hovertemplate='x: %{x} <br>y: %{y}') )
fig.update_layout(title="Regression Results",hovermode="x")
fig.show()

# Load Classification
names = list({"bianca":0,"gialla": 1, "arancione": 2, "rossa": 3})
C_model = ClassificationNet(num_inputs=31,num_classes=4).to(device)
C_weights = torch.load("weights_classication.pt")
C_model.load_state_dict(C_weights)
C_model = C_model.to(device)

# Predict Classication
C_Xt = torch.from_numpy(C_X).type(torch.float32).unsqueeze(1)
C_Xt = (C_Xt - C_mean) / C_std
C_Y_hat = C_model.forward(C_Xt).argmax(dim=-1,keepdim=True).detach().numpy().reshape(-1)

# Visualize results
cm = confusion_matrix(C_Y,C_Y_hat)
names_pred = [ "Pred: " + n for n in names]
df = pd.DataFrame(cm, columns=names_pred, index=names)
print(df)



           Pred: bianca  Pred: gialla  Pred: arancione  Pred: rossa
bianca              877            33               29            4
gialla               14           853                6            0
arancione            27            15              639            1
rossa                 1             0                1          188


In [ ]:
# Print Regression weights
R_layers=[x.data for x in R_model.parameters()] 

# Print Classification weights 
C_layers=[x.data for x in R_model.parameters()] 

print(R_layers)
print("-----------------")
print(C_layers)

## From Pytorch to Excel

In [109]:
import numpy as np

def relu(x):
  return np.maximum(0,x)

# Usa gli input di prima oppure dei nuovi
data = {
    "input" : [0,4,12,24,32,40,52,61,70,74,83,92,100,110,119],
    "output" : [1,1,1,1.5,2,3,5,7,9,10,13,16,20,25,29]
}

# Input
x = np.array(data["input"])

# Normalization
x = (x -np.mean(x))/np.std(x)

# Output
y = np.array(data["output"])

w_h1 = np.array([[ 1.5185],
                  [ 1.4294],
                  [-1.3231],
                  [ 0.6863],
                  [ 1.3232],
                  [ 1.4310]]).T
b_h1 = np.array( [[-0.7579,  1.2593,  0.0897,  0.5494,  0.0047, -1.2937]] )
w_h2 = np.array([[-0.3675, -0.0741, -0.2361, -0.0342, -1.0449, -0.2139],
                  [ 1.2759,  0.9136, -0.5146,  0.2920,  0.8433,  1.1550],
                  [ 0.8940,  0.9680, -0.7654,  0.4420,  1.0525,  0.8990],
                  [ 0.9814,  0.9840, -0.0711,  0.1270,  1.1307,  0.9849]]).T
b_h2 = np.array( [[0.2226, 0.9810, 0.0962, 0.8358]])
w_h3 = np.array( [ [-1.0961,  0.9169,  1.0135,  1.0918] ]).T
b_h3 = np.array( [[0.2489]] )

v1 = relu(np.dot(x.reshape(-1,1),w_h1) + b_h1 )
v2 = relu(np.dot(v1, w_h2) + b_h2)
y_hat = (np.dot(v2,w_h3) + b_h3 ).reshape(-1)


# Deploy
fig = go.Figure()
fig.add_traces( go.Scatter(x=x, y=y, name="Real"))
fig.add_traces( go.Scatter(x=x, y=y_hat, name="Estimate"))
fig.show()


#layers=[x.data for x in md.parameters()] 
#layers
